In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd drive/MyDrive/Cardiac\ Project/Registration_phase/based\ on\ doc2

/content/drive/MyDrive/Cardiac Project/Registration_phase/based on doc2


In [3]:
import cv2
import os
import glob
from tqdm import tqdm
import pandas as pd
import json
import numpy as np

In [4]:
!pip install pydicom -q

In [5]:
import glob
from tqdm import tqdm
import pydicom
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

In [6]:
def convert_dcm2png(Main_path):
    i = 0
    for path in tqdm(glob.iglob(Main_path + '/**/*.dcm' , recursive=True) , desc = "Reading"):
        i+=1
        DCM_file = pydicom.dcmread(path) # dcmread read data as array with size (H * W) and dtype uint16
        image = DCM_file.pixel_array  
        shape = image.shape
        plt.show()
        image_2d = image.astype(float)
        image_2d_scaled = (np.maximum(image_2d,0) / image_2d.max()) * 255.0
        image_2d_scaled = np.uint8(image_2d_scaled)

        destination = path[:-3] + 'png'
        #print('image number:' , i , "| name:" , path , '|convert to:' , destination)
 
        with open(destination, 'wb') as png_file:
            cv2.imwrite(destination ,  cv2.cvtColor(image_2d_scaled, cv2.COLOR_RGB2BGR) )
             
        os.remove(path)

In [7]:
def Extract_bb_class(meta_data, image_name):
    
    # extract Label column
    this_image_Label = json.loads(meta_data[meta_data['External ID'] == image_name].Label.iloc[0])    
    # Bounding Box
    bbox_ax = this_image_Label['objects'][0]['bbox']          
    # Class Type
    class_type = this_image_Label['classifications'][0]['answer']['value']

    return bbox_ax, class_type

In [8]:
def crop_this(image , bbox_ax):
    cropped_image = image[bbox_ax['top']:bbox_ax['top']+bbox_ax['height'], bbox_ax['left']:bbox_ax['left']+bbox_ax['width']]
    shape = cropped_image.shape
    image_2d = cropped_image.astype(float)
    image_2d_scaled = (np.maximum(image_2d,0) / image_2d.max()) * 255.0
    image_2d_scaled = np.uint8(image_2d_scaled)
    return image_2d_scaled

In [9]:
def crop(main_path):
    patient_id = main_path.split("/")[-1]
    meta_data_path = main_path+"/"+patient_id+".csv"
    meta_data = pd.read_csv(meta_data_path)

    files = os.listdir(main_path)
    folders = [f for f in files if len(f) == 1]
    for folder in folders:
        folder_path = main_path + "/" + folder
        fs = os.listdir(folder_path)
        for f in fs:
            if "body" in f:
                cine_seri_folder = folder_path+"/"+f

            if ".png" in f and "unknown" not in f:
                bb , class_type = Extract_bb_class(meta_data, f)
                if class_type == "cine":
                    this_bb = bb
        
        cine_seri_destination = folder_path + "/cine_seri"
        if not os.path.exists(cine_seri_destination):
            os.mkdir(cine_seri_destination)

        cine_images = os.listdir(cine_seri_folder)
        for cine_image_name in cine_images:
            cine_image_path = cine_seri_folder + "/" + cine_image_name
            cine_image = cv2.imread(cine_image_path , cv2.IMREAD_GRAYSCALE)
            cropped_cine_image = crop_this(cine_image , this_bb)
            destination = cine_seri_destination + "/" + cine_image_name
            with open(destination, 'wb') as png_file:
                cv2.imwrite(destination , cropped_cine_image)


In [12]:
patient_path = "dataset/ok1,2/510347"
convert_dcm2png(patient_path)


Reading: 0it [00:00, ?it/s]


In [13]:
crop(patient_path)